In [1]:
import Crypto
from Crypto.PublicKey import RSA
from Crypto import Random
import base64
from Crypto.Cipher import PKCS1_OAEP

from Crypto.Signature import pkcs1_15
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding


from Crypto.Hash import SHA512, SHA384, SHA256, SHA, MD5
from base64 import b64encode, b64decode
import rsa



In [2]:
hash = "SHA-256"

def newkeys(keysize):
    random_generator = Random.new().read
    key = RSA.generate(keysize, random_generator)
    private, public = key, key.publickey()
    return public, private



In [3]:
def importKey(externKey):
    return RSA.importKey(externKey)

def getpublickey(priv_key):
    return priv_key.publickey()

def encrypt(message, pub_key):
    #RSA encryption protocol according to PKCS#1 OAEP
    cipher = PKCS1_OAEP.new(pub_key)
    return cipher.encrypt(message)



In [4]:
def decrypt(ciphertext, priv_key):
    #RSA encryption protocol according to PKCS#1 OAEP
    cipher = PKCS1_OAEP.new(priv_key)
    return cipher.decrypt(ciphertext)



In [5]:
def sign(message, priv_key, hashAlg="SHA-256"):
    global hash
    hash = hashAlg
    signer = PKCS1_v1_5.new(priv_key)
    if (hash == "SHA-512"):
        digest = SHA512.new()
    elif (hash == "SHA-384"):
        digest = SHA384.new()
    elif (hash == "SHA-256"):
        digest = SHA256.new()
    elif (hash == "SHA-1"):
        digest = SHA.new()
    else:
        digest = MD5.new()
    digest.update(message)
    return signer.sign(digest)



In [6]:
def verify(message, signature, pub_key):
    signer = PKCS1_v1_5.new(pub_key)
    if (hash == "SHA-512"):
        digest = SHA512.new()
    elif (hash == "SHA-384"):
        digest = SHA384.new()
    elif (hash == "SHA-256"):
        digest = SHA256.new()
    elif (hash == "SHA-1"):
        digest = SHA.new()
    else:
        digest = MD5.new()
    digest.update(message)
    return signer.verify(digest, signature)
    

In [8]:
(public, private) = rsa.newkeys(2048)


In [9]:

def sender():
    text="Hello this is sender! my port is : "
    port = int(input("enter port number to communicate"))
    keysize = 2048
    msg = text + str(port)
    msg = bytes(msg, 'utf-8')
    packet = b64encode(rsa.encrypt(msg, public))
    sender_sign = b64encode(rsa.sign(msg, private, "SHA-512"))    
    return packet,sender_sign,port
    
    
    

In [ ]:
def firewall():
    accepted_ports = [21,777,80,20]
    #port=20
    while True:
        (p,ss,port) = sender()
        if port in accepted_ports:
            rec(p,ss)
        else:
            print("Intrusion from different port")
        
    

In [ ]:
def rec(packet,sender_sign):
    decrypted = rsa.decrypt(b64decode(packet), private)
    print(decrypted)

    verify = rsa.verify(decrypted, b64decode(sender_sign), public)
    print("the digital signature matches that of sender, the hash: "+str(verify))
    
    

In [ ]:
p,ss,port = sender()

In [ ]:
firewall()